In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI

# ✅ 1. 환경변수 로드 및 GPT 클라이언트 설정
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# ✅ 2. 정량 분석 함수 정의
def quantify_youtube_metrics(df, label):
    df = df.copy()
    df['comment_len'] = df['comment'].astype(str).apply(len)
    df['counts'] = pd.to_numeric(df['counts'], errors='coerce')
    df['likes_count'] = pd.to_numeric(df['likes_count'], errors='coerce')
    df['comment_count'] = pd.to_numeric(df['comment_count'], errors='coerce')

    return {
        '분석 대상': label,
        '영상 수': df['videoId'].nunique(),
        '전체 댓글 수': len(df),
        '평균 댓글 길이': round(df['comment_len'].mean(), 2),
        '평균 댓글 수(영상별)': round(df.groupby('videoId')['comment'].count().mean(), 2),
        '평균 조회수': round(df.groupby('videoId')['counts'].first().mean(), 2),
        '평균 좋아요 수': round(df.groupby('videoId')['likes_count'].first().mean(), 2),
        '평균 댓글 수(메타 기준)': round(df.groupby('videoId')['comment_count'].first().mean(), 2)
    }

# ✅ 3. GPT 요약 함수
def generate_gpt_summary(summary_kim, summary_lee):
    prompt = f"""
다음은 두 정치인의 유튜브 통계 분석 결과입니다. 주요 차이점을 중심으로 비교 요약을 작성해 주세요.

[김문수]
{summary_kim}

[이재명]
{summary_lee}

요약 방식:
- 어떤 후보가 어떤 지표에서 우세한지 기술
- 수치 인용 포함
- 콘텐츠 반응 특성 해석 포함
"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4
    )
    return response.choices[0].message.content.strip()

# ✅ 4. 파일 경로 지정
kim_file = "youtube_통합분석_김문수_후보_등록.csv"
lee_file = "youtube_통합분석_이재명_전남_방문.csv"

# ✅ 5. 데이터 로딩 및 분석
df_kim = pd.read_csv(kim_file)
df_lee = pd.read_csv(lee_file)

summary_kim = quantify_youtube_metrics(df_kim, '김문수')
summary_lee = quantify_youtube_metrics(df_lee, '이재명')
df_summary = pd.DataFrame([summary_kim, summary_lee])

# ✅ 6. 저장 + GPT 요약 생성
df_summary.to_csv("youtube_정량분석_요약.csv", index=False, encoding='utf-8-sig')

gpt_text = generate_gpt_summary(summary_kim, summary_lee)

# ✅ 7. 미리보기 + 저장 여부 확인
print("\n📋 GPT 요약 미리보기:")
print("=" * 60)
print(gpt_text)
print("=" * 60)

save = input("📥 이 요약을 저장할까요? (y/n): ").lower()
if save == "y":
    with open("youtube_정량분석_해설.txt", "w", encoding="utf-8") as f:
        f.write(gpt_text)
    print("✅ 저장 완료 → youtube_정량분석_해설.txt")
else:
    print("❌ 저장하지 않았습니다.")



📋 GPT 요약 미리보기:
김문수와 이재명의 유튜브 통계를 비교해보면, 이재명이 평균 조회수는 35,405.97로 김문수의 44,798.63보다 낮지만, 평균 좋아요 수는 2,342.53으로 김문수의 1,090.67보다 높습니다. 이재명의 평균 댓글 수(메타 기준)는 111.06으로 김문수의 40.19보다 훨씬 높은 반면, 김문수의 평균 댓글 수(영상별)는 21.8로 이재명의 28.09보다 낮습니다. 이를 통해 이재명의 영상이 더 많은 관심과 참여를 유도하고 있으며, 김문수의 영상은 더 많은 영상 수를 업로드하고 있음을 알 수 있습니다.
✅ 저장 완료 → youtube_정량분석_해설.txt
